In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Current time (assuming it's the time when the model is being run)
current_time = pd.Timestamp.now()

# Calculate ideal time to delivery for each order
order_df['Ideal_Time'] = (order_df['Expected Time of Arrival'] - current_time).dt.total_seconds() / 3600  # in hours

# Calculate time T_j for each ship
ship_df['T_j'] = (ship_df['Destination Time'] - ship_df['Arrival Time']).dt.total_seconds() / 3600  # in hours

# Define the model
m = gp.Model("port_logistics")

# Number of orders and ships
num_orders = len(order_df)
num_ships = len(ship_df)

# Decision Variables
x = m.addVars(num_orders, num_ships, vtype=GRB.BINARY, name="x")  # x[i,j]
y = m.addVars(num_ships, lb=0, ub=1, name="y")  # y[j]
t = m.addVars(num_orders, lb=0, name="t")  # t[i]
z = m.addVars(num_orders, num_orders, num_ships, vtype=GRB.BINARY, name="z")  # z[i,k,j]

# Objective Function
m.setObjective(gp.quicksum(y[j] for j in range(num_ships)) - 
               gp.quicksum(order_df.loc[i, 'Weight of Order (tons)'] * 
                           (2 if ship_df.loc[j, 'Priority'] == 'Express' else 1) * t[i] 
                           for i in range(num_orders) for j in range(num_ships)), GRB.MAXIMIZE)

# Constraints
# 1. Assignment Constraint
for i in range(num_orders):
    m.addConstr(gp.quicksum(x[i, j] for j in range(num_ships)) == 1)

# 2. Capacity Constraint
for j in range(num_ships):
    m.addConstr(gp.quicksum(order_df.loc[i, 'Weight of Order (tons)'] * x[i, j] for i in range(num_orders)) <= ship_df.loc[j, 'Vessel Capacity (tonnes)'])

# 3. Fullness Definition
for j in range(num_ships):
    m.addConstr(y[j] == gp.quicksum(order_df.loc[i, 'Weight of Order (tons)'] * x[i, j] for i in range(num_orders)) / ship_df.loc[j, 'Vessel Capacity (tonnes)'])

# 4. Combination Constraint
for i in range(num_orders):
    for k in range(num_orders):
        for j in range(num_ships):
            m.addConstr(x[i, j] - x[k, j] <= 1 - z[i, k, j])
            m.addConstr(x[k, j] - x[i, j] <= 1 - z[i, k, j])

# 5. Time Constraint
for i in range(num_orders):
    m.addConstr(t[i] >= order_df.loc[i, 'Ideal_Time'] - gp.quicksum(x[i, j] * ship_df.loc[j, 'T_j'] for j in range(num_ships)))

# Model assembly complete. Let's return a confirmation for now.
"Optimization model has been set up."
